In [1]:
import psycopg2

In [73]:
connection = psycopg2.connect(
      database="InformationResearch", 
      user="postgres", 
      password="qwerty123", 
      host="localhost"
    )
cur = connection.cursor()

In [12]:
cur.execute("SELECT to_tsvector('The quick brown fox jumped over the lazy dog.')")
cur.fetchall()

[("'brown':3 'dog':9 'fox':4 'jump':5 'lazi':8 'quick':2",)]

In [16]:
cur.execute("SELECT to_tsvector('russian', 'Съешь ещё этих мягких французских булок, да выпей чаю')")
cur.fetchall()

[("'булок':6 'вып':8 'ещ':2 'мягк':4 'съеш':1 'французск':5 'ча':9 'эт':3",)]

Почему в векторе нет слова да? Потому что да - предлог, а предлог не может быть лексемой.

In [20]:
cur.execute("SELECT to_tsvector('The quick brown fox jumped over the lazy dog') @@ to_tsquery('fox')")
cur.fetchall()

[(True,)]

In [21]:
cur.execute("SELECT to_tsvector('The quick brown fox jumped over the lazy dog') @@ to_tsquery('foxes')")
cur.fetchall()

[(True,)]

In [22]:
cur.execute("SELECT to_tsvector('The quick brown fox jumped over the lazy dog') @@ to_tsquery('foxhound')")
cur.fetchall()

[(False,)]

@@ - оператор соответствия, который возвращает true, если tsvector (документ) соответствует tsquery (запросу). Для этого оператора не важно, какой тип записан первым.

Почему второй запрос возвращает true, а третий не возвращает? Потому что во втором запросе fox - лексема слова foxes, а в третьем fox не является лексемой слова foxhound (fox - лиса, foxhound - гончая).

In [20]:
cur.execute("SELECT to_tsvector('Russian', 'Съешь ещё этих мягких французских булок, да выпей чаю.') @@ to_tsquery('Russian','булка')")
cur.fetchall()

[(False,)]

In [24]:
cur.execute("SELECT ts_lexize('russian_stem', 'булок')")
cur.fetchall()

[(['булок'],)]

In [25]:
cur.execute("SELECT ts_lexize('russian_stem', 'булка')")
cur.fetchall()

[(['булк'],)]

Почему слово булка не находится? Потому что для нее нет лексем.

In [13]:
cur.execute("SELECT to_tsvector('Russian', 'Съешь ещё этих мягких французских пирожков, да выпей чаю.') @@ to_tsquery('Russian','пирожки')")
cur.fetchall()

[(True,)]

In [14]:
cur.execute("SELECT to_tsvector('Russian', 'Съешь ещё этих мягких французских пирожков, да выпей чаю.') @@ to_tsquery('Russian','пирожок')")
cur.fetchall()

[(False,)]

Почему первый запрос возвращает true, а второй не возвращает? Потому что для слова "пирожки" лексема - "пирожк" и она присутствует в первом запросе, а для слова "пирожок" лексема - "пирожок".

In [30]:
cur.execute("SELECT to_tsvector('The quick brown fox jumped over the lazy dog') @@ to_tsquery('fox & dog')")
cur.fetchall()

[(True,)]

In [31]:
cur.execute("SELECT to_tsvector('The quick brown fox jumped over the lazy dog') @@ to_tsquery('fox | rat')")
cur.fetchall()

[(True,)]

In [32]:
cur.execute("SELECT to_tsvector('The quick brown fox jumped over the lazy dog') @@ to_tsquery('!clown')")
cur.fetchall()

[(True,)]

In [33]:
cur.execute("SELECT to_tsvector('The quick brown fox jumped over the lazy dog') @@ to_tsquery('fox & (dog | rat) & !mice')")
cur.fetchall()

[(True,)]

In [34]:
cur.execute("SELECT to_tsvector('russian', 'Быстрая коричневая лиса перепрыгнула через ленивую собаку') @@ to_tsquery('лиса & собака')")
cur.fetchall()

[(True,)]

In [35]:
cur.execute("SELECT to_tsvector('russian', 'Быстрая коричневая лиса перепрыгнула через ленивую собаку') @@ to_tsquery('лиса | крыса')")
cur.fetchall()

[(True,)]

In [37]:
cur.execute("SELECT to_tsvector('russian', 'Быстрая коричневая лиса перепрыгнула через ленивую собаку') @@ to_tsquery('!клоун')")
cur.fetchall()

[(True,)]

In [38]:
cur.execute("SELECT to_tsvector('russian', 'Быстрая коричневая лиса перепрыгнула через ленивую собаку') @@ to_tsquery('лиса&(собака|крыса)&!мышь')")
cur.fetchall()

[(True,)]

Почему для английского языка не нужно указывать язык в первом аргументе и какой анализатор используется если никакой не указан? Потому что по умолчанию стоит английский анализатор. Если никакой анализатор не указан, то берется анализатор по умолчанию.

In [40]:
cur.execute("SELECT to_tsvector('Russian', 'Съешь ещё этих мягких французских булок, да выпей чаю.') @@ to_tsquery('Russian','мягких<2>булок')")
cur.fetchall()

[(True,)]

Что означает число 2 в операторе <->? <-> (предшествует) - оператор, который находит соответствие, только если его операнды расположены рядом и в заданном порядке. Число 2 допускает существование ровно одной лексемы между этими лексемами.

In [41]:
cur.execute("SELECT to_tsvector('Russian', 'Съешь ещё этих мягких французских булок, да выпей чаю.') @@ to_tsquery('Russian','съешь<->еще')")
cur.fetchall()

[(True,)]

Для чего нужно использовать функцию phraseto_tsquery? Функция phraseto_tsquery задействует этот оператор для конструирования tsquery, который может содержать многословную фразу, включающую в себя стоп-слова. 

In [64]:
cur.execute("SELECT ts_debug('Russian', 'Съешь ещё этих мягких французских булок, да выпей чаю.')")
cur.fetchall()

[('(word,"Word, all letters",Съешь,{russian_stem},russian_stem,{съеш})',),
 ('(blank,"Space symbols"," ",{},,)',),
 ('(word,"Word, all letters",ещё,{russian_stem},russian_stem,{ещ})',),
 ('(blank,"Space symbols"," ",{},,)',),
 ('(word,"Word, all letters",этих,{russian_stem},russian_stem,{эт})',),
 ('(blank,"Space symbols"," ",{},,)',),
 ('(word,"Word, all letters",мягких,{russian_stem},russian_stem,{мягк})',),
 ('(blank,"Space symbols"," ",{},,)',),
 ('(word,"Word, all letters",французских,{russian_stem},russian_stem,{французск})',),
 ('(blank,"Space symbols"," ",{},,)',),
 ('(word,"Word, all letters",булок,{russian_stem},russian_stem,{булок})',),
 ('(blank,"Space symbols",", ",{},,)',),
 ('(word,"Word, all letters",да,{russian_stem},russian_stem,{})',),
 ('(blank,"Space symbols"," ",{},,)',),
 ('(word,"Word, all letters",выпей,{russian_stem},russian_stem,{вып})',),
 ('(blank,"Space symbols"," ",{},,)',),
 ('(word,"Word, all letters",чаю,{russian_stem},russian_stem,{ча})',),
 ('(blank,

In [65]:
cur.execute("SELECT ts_headline('Russian', 'Съешь ещё этих мягких французских булок, да выпей чаю.', to_tsquery('Russian', 'Мягкие'))")
cur.fetchall()

[('Съешь ещё этих <b>мягких</b> французских булок, да выпей чаю.',)]